In [1]:
import pandas as pd
import numpy as np
import glob

import pyarrow as pa
import pyarrow.dataset as ds

In [2]:
businesses_df = pd.read_json("yelp_dataset/yelp_academic_dataset_business.json", lines=True)

In [3]:
reader = pd.read_json(
    "yelp_dataset/yelp_academic_dataset_review.json",
    lines=True,
    chunksize=200_000,
    dtype_backend="pyarrow"
)

for i, chunk in enumerate(reader):
    chunk.to_parquet(
        f"yelp_dataset_parquet/reviews_part_{i:04}.parquet",
        index=False
    )

In [4]:
reader2 = pd.read_json(
    "yelp_dataset/yelp_academic_dataset_review.json",
    lines=True,
    chunksize=5_000
)
sample = next(reader2)
col_names = sample.columns.tolist()
print(col_names)

['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny', 'cool', 'text', 'date']


In [5]:
reviews_dataset = ds.dataset("yelp_dataset_parquet", format="parquet")
sorted_file_paths = sorted(glob.glob("yelp_dataset_parquet/reviews_part_*.parquet"))
reviews_df = pd.concat([pd.read_parquet(p, columns=col_names) for p in sorted_file_paths], ignore_index=True)

In [6]:
fast_food_df = pd.read_csv("FastFoodRestaurantsLocations.csv")
fast_food_names_in_region = fast_food_df.loc[fast_food_df["province"] == "FL", "name"].unique().tolist()

In [7]:
businesses_in_FL_df = businesses_df.loc[businesses_df["state"] == "FL"]
fast_food_business_in_fl_df = businesses_in_FL_df.loc[businesses_in_FL_df["name"].isin(fast_food_names_in_region)]

In [8]:
ff_businesses_id_fl = fast_food_business_in_fl_df["business_id"].unique().tolist()

In [9]:
# businesses IDs of fast food restaurants in Florida
fast_food_business_in_fl_df["business_id"].nunique()

871

In [10]:
fast_food_business_reviews_fl_df = reviews_df.loc[reviews_df["business_id"].isin(ff_businesses_id_fl)]

In [11]:
print(fast_food_business_reviews_fl_df.shape)
print(fast_food_business_reviews_fl_df["business_id"].nunique())

(21149, 9)
871


In [12]:
fast_food_business_reviews_fl_df.to_csv("output/Florida_Fast_Food_Reviews.csv", index=False)
fast_food_business_in_fl_df.to_csv("output/Florida_Fast_Food_Businesses.csv", index=False)

In [13]:
pd.

SyntaxError: invalid syntax (3133375982.py, line 1)